In [1]:
import os
import vllm
import json
from easysteer.hidden_states import get_all_hidden_states
from vllm import LLM

from easysteer.steer import extract_pca_control_vector,StatisticalControlVector


os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["VLLM_USE_V1"] = "0"



model_path = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"

# vector_path = "vectors/thinking_switch_pca_MATH-500.gguf" #MATH-500


num_question = 20

/home/meixinyu/anaconda3/envs/easysteer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-13 22:12:29 [__init__.py:244] Automatically detected platform cuda.


In [2]:
file_path = "../../temp/test.jsonl" #MATH-500
# file_path = "../../temp/test_gsm8k.jsonl" #GSM8K

vector_path = "MATH500.gguf"

In [3]:
problem_list = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        line_count = 0
        for line in f:
            if line_count >= num_question:
                break
            stripped_line = line.strip()
            if not stripped_line:
                continue
                
            try:
                data = json.loads(stripped_line)
                if "problem" in data:
                    problem_list.append(data["problem"])
                    line_count += 1
                elif "question" in data:
                    problem_list.append(data["question"])
                    line_count += 1
            except json.JSONDecodeError:
                print(f"skip: {line[:50]}...")
                continue

except Exception as e:
    print(f"{str(e)}")
# problem_list = ["Find the roots of $(x - 3)^3 + (x -7)^3 = (2x - 10)^3.$","A regular hexagon can be divided into six equilateral triangles. If the perimeter of one of the triangles is 21 inches, what is the perimeter, in inches, of the regular hexagon?",""]

In [4]:
slow_thinking = [
    "Walk me through your complete reasoning process step by step. Output as many procedures as you can."
]

fast_thinking = [
    "Direct answer only."
]

len_slow=len(slow_thinking)
len_fast=len(fast_thinking)

In [5]:
texts = []

for prob in problem_list:
    for req in slow_thinking:
        texts.append(f"""<|im_start|>user
{prob}{req}<|im_end|>
<|im_start|>assistant
""")

for prob in problem_list:
    for req in fast_thinking:
        texts.append(f"""<|im_start|>user
{prob}{req}<|im_end|>
<|im_start|>assistant
""")
print(texts[0])
print(texts[num_question*len_slow-1])
print(texts[num_question*len_slow])
with open("111.txt", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(text + "\n")  # 每个元素后加换行符


<|im_start|>user
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$Walk me through your complete reasoning process step by step. Output as many procedures as you can.<|im_end|>
<|im_start|>assistant

<|im_start|>user
Let $a$ be a positive real number such that all the roots of
\[x^3 + ax^2 + ax + 1 = 0\]are real.  Find the smallest possible value of $a.$Walk me through your complete reasoning process step by step. Output as many procedures as you can.<|im_end|>
<|im_start|>assistant

<|im_start|>user
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$Direct answer only.<|im_end|>
<|im_start|>assistant



In [6]:
llm = LLM(model=model_path,task="reward",tensor_parallel_size=1)

all_hidden_states, outputs = get_all_hidden_states(llm, texts)

INFO 07-13 22:12:38 [config.py:1472] Using max model len 32768


2025-07-13 22:12:38,465	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-13 22:12:38 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-13 22:12:38 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.08it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.08it/s]


INFO 07-13 22:12:41 [default_loader.py:272] Loading weights took 1.00 seconds


INFO 07-13 22:12:41 [model_runner.py:1255] Model loading took 2.8876 GiB and 1.150251 seconds


Processed prompts: 100%|██████████| 40/40 [00:00<00:00, 58.66it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


In [7]:

control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states,
    positive_indices=list(range(num_question*len_slow)), 
    negative_indices=list(range(num_question*len_slow,num_question*(len_slow+len_fast))),
    model_type="qwen2.5",
    token_pos=-1,
    normalize=True
)

Computing PCA directions:   0%|          | 0/28 [00:00<?, ?it/s]

Computing PCA directions: 100%|██████████| 28/28 [00:00<00:00, 531.46it/s]


In [8]:
control_vector.export_gguf(vector_path)
control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([-0.01113699, -0.06029188,  0.04215517, ..., -0.01189059,
        -0.02577175, -0.01083255], shape=(1536,), dtype=float32), 1: memmap([ 0.02075267, -0.00759126,  0.01752456, ..., -0.01258039,
        -0.03163083, -0.04065328], shape=(1536,), dtype=float32), 2: memmap([ 0.0100673 , -0.0234082 ,  0.02484847, ...,  0.06437913,
        -0.01141085, -0.05111764], shape=(1536,), dtype=float32), 3: memmap([-0.01157416, -0.00161835,  0.04929598, ...,  0.02504394,
        -0.02055443, -0.04532089], shape=(1536,), dtype=float32), 4: memmap([ 0.00013692,  0.01033693,  0.02944114, ...,  0.03095547,
        -0.03417241, -0.00300148], shape=(1536,), dtype=float32), 5: memmap([-0.00319834,  0.04855272,  0.02122804, ..., -0.01298449,
        -0.04043267, -0.03302082], shape=(1536,), dtype=float32), 6: memmap([ 0.01014982,  0.00519793,  0.03920706, ..., -0.01955185,
         0.04191752,  0.01931267], shape=(1536,), dtype